In [1]:
from yggdrasil.examples import yamls as ex_yamls
import yaml
import jupyterlab_nodeeditor as jlne
import pytest

In [58]:
# Improved version of making a JLNE-compliant dictionary from a Yggdrasil Model YAML
# Still semi-hard coded for the Photosynthesis model -- WIP on lessening the hardcode, but there is a Ygg pattern that follows
# TO-DO : Make more robust, Test on custom models 1-3
def dict_conversion(model_file):
    # Using flags here in case I need them later, will remove if I don't
    is_dict, is_list = False, False
    
    # If the type of the safe load is a dictionary, we know it is a single model
    # If it is a list, then we have multiple models
    # If it's neither, return the type so I can work on it
    if type(model_file) is dict:
        is_dict = True
    elif type(model_file) is list:
        is_list = True
    else:
        return ("Unaccounted model type: ", type(model_file))
    
    # Let's do everything all at once, can split it up if needed
    # Check if it's a single model (dictionary) or multiple (list)
    if is_dict:
        # Setup initial dictionary to be filled
        new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], []
        new_dict["title"] = model_file["name"]
        
        # Fill in the Inputs
        if "input" in model_file.keys():
            for i, inp in enumerate(model_file["input"]):
                print(i, inp)
                new_dict["inputs"].append({'title': inp["name"], 'key': "temp_in" + str(i), 'socket_type': inp["default_file"]["filetype"]})
        elif "inputs" in model_file.keys():
            for i, inp in enumerate(model_file["inputs"]):
                new_dict["inputs"].append({'title': inp["name"], 'key': "temp_in" + str(i), 'socket_type': inp["default_file"]["filetype"]})
        
        # Fill in the Outputs, same as inputs with name changes
        if "output" in model_file.keys():
            for o, out in enumerate(model_file["output"]):
                new_dict["outputs"].append({'title': out["name"], 'key': "temp_out" + str(o), 'socket_type': out["default_file"]["filetype"]})
        elif "outputs" in model_file.keys():
            for o, out in enumerate(model_file["outputs"]):
                new_dict["outputs"].append({'title': out["name"], 'key': "temp_out" + str(o), 'socket_type': out["default_file"]["filetype"]})
        
        # We return a list here so that way loading the models is uniform for both multi-model and single-model files
        return [new_dict]
    

    # We can pretty much do the dictionary loop multiple times per model then return the list
    elif is_list:
        model_list = []
        for model in model_file:
            # Setup initial dictionary to be filled
            new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], []
            new_dict["title"] = model["name"]
        
            # Fill in the Inputs
            if "input" in model.keys():
                for i, inp in enumerate(model["input"]):
                    new_dict["inputs"].append({'title': inp["name"], 'key': "temp_in" + str(i), 'socket_type': inp["default_file"]["filetype"]})
            elif "inputs" in model.keys():
                for i, inp in enumerate(model["inputs"]):
                    print(i, inp)
                    new_dict["inputs"].append({'title': inp["name"], 'key': "temp_in" + str(i), 'socket_type': inp["default_file"]["filetype"]})
        
            # Fill in the Outputs, same as inputs with name changes
            if "output" in model.keys():
                for o, out in enumerate(model["output"]):
                    new_dict["outputs"].append({'title': out["name"], 'key': "temp_out" + str(o), 'socket_type': out["default_file"]["filetype"]})
            elif "outputs" in model.keys():
                for o, out in enumerate(model["outputs"]):
                    new_dict["outputs"].append({'title': out["name"], 'key': "temp_out" + str(o), 'socket_type': out["default_file"]["filetype"]})
                    
            model_list.append(new_dict)
        
        # Return the full converted list of models
        return model_list

In [59]:
load_model()

Please enter filepath for model or leave blank for an example:  sample_models/sample_2.yml


0 {'name': 'input', 'default_file': {'name': './Input/input.txt', 'filetype': 'table'}}
[{'inputs': [{'title': 'input', 'key': 'temp_in0', 'socket_type': 'table'}], 'outputs': [{'title': 'output', 'key': 'temp_out0', 'socket_type': 'table'}], 'title': 'Sample2'}]


In [61]:
with open("sample_models/sample_1.yml", "r") as sample:
    sample_model = yaml.safe_load(sample)["models"]
    print(type(sample_model))
    print(sample_model)
    print(sample_model[0])

<class 'list'>
[{'name': 'modelA', 'language': 'python', 'args': './src/modelA.py', 'inputs': 'inputA', 'outputs': 'outputA'}, {'name': 'modelB', 'language': 'python', 'args': './src/modelB.py', 'inputs': 'inputB', 'outputs': 'outputB'}, {'name': 'modelC', 'language': 'python', 'args': './src/modelC.py', 'inputs': 'inputC', 'outputs': 'outputC'}]
{'name': 'modelA', 'language': 'python', 'args': './src/modelA.py', 'inputs': 'inputA', 'outputs': 'outputA'}


In [38]:
# Testing Functions

# The first test is just to visually ensure that the model appears in the node editor instance
# This is NOT an explicit function
# Look for Inputs, Outputs, and a proper title

# Test 2 : Verify Editor instance w output
# This is done with JLNE's built in class methods of calling a unique editor instance with the models
def test_instance(test_editor):
    expected_editor = jlne.NodeEditor()
    assert test_editor == expected_editor

# Test 3 : Blank

In [60]:
# By default, load the photosynthesis model
# ps - Node Editor instance that it is added to, default blank
def load_model(ps = None):
    # TODO: Here is where it can be "model" or "models"
    # E.g. filepath: sample_models/sample_2.yml
    filepath = input("Please enter filepath for model or leave blank for an example: ")
    fsample = None
    if filepath:
        with open(filepath, "r") as sample:
            sample_model = yaml.safe_load(sample)
            if "model" in sample_model.keys():
                fsample = sample_model["model"]
            elif "models" in sample_model.keys():
                fsample = sample_model["models"]
    else:
        with open(ex_yamls['fakeplant']['python'], "r") as sample:
            fsample = yaml.safe_load(sample)["model"]
        
    ne_instance = ps or jlne.NodeEditor()
    model_list = dict_conversion(fsample)
    
    # Print for debugging
    print(model_list)
    
    for model in model_list:
        ne_instance.add_component(model)
    
    return ne_instance